In [ ]:
!wget -r -np -R index.html "https://a68c-126-170-21-44.ngrok-free.app/Deep-Emotion/kaggle_dataset_v2.zip"

!mv ./a68c-126-170-21-44.ngrok-free.app/Deep-Emotion/kaggle_dataset_v2.zip ./

!unzip ./kaggle_dataset_v2.zip

!pip install opacus

!pip install wandb

!wget -r -np -R index.html "https://a68c-126-170-21-44.ngrok-free.app/Deep-Emotion/Expw.zip"

!mv ./a68c-126-170-21-44.ngrok-free.app/Deep-Emotion/Expw.zip ./

!unzip ./Expw.zip

!pip install imgaug

串流輸出內容已截斷至最後 5000 行。
  inflating: Expw-F/train/sad/34026distaste_yang_374.jpg  
  inflating: Expw-F/train/sad/34535crying_asian_234.jpg  
  inflating: Expw-F/train/sad/37602crying_mother_475.jpg  
  inflating: Expw-F/train/sad/17627frightened_by_the_movie_chinese_639.jpg  
  inflating: Expw-F/train/sad/27684crying_boss_258.jpg  
  inflating: Expw-F/train/sad/36470crying_grandfather_440.jpg  
  inflating: Expw-F/train/sad/34804crying_boss_248.jpg  
  inflating: Expw-F/train/sad/34550crying_asian_283.jpg  
  inflating: Expw-F/train/sad/44278heartbroken_man_281.jpg  
  inflating: Expw-F/train/sad/30030heartbroken_family_230.jpg  
  inflating: Expw-F/train/sad/37132crying_lady_387.jpg  
  inflating: Expw-F/train/sad/28331crying_man_118.jpg  
  inflating: Expw-F/train/sad/44999heartbroken_programmer_418.jpg  
  inflating: Expw-F/train/sad/87129awe_actor_61.jpg  
  inflating: Expw-F/train/sad/28262crying_infant_336.jpg  
  inflating: Expw-F/train/sad/28517crying_old_58.jpg  
  inflating: Exp

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import torchvision.transforms as transforms
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
import torch.nn.functional as F
import torchvision.models as models
from torch.optim.lr_scheduler import StepLR, ReduceLROnPlateau
from opacus.validators import ModuleValidator
from opacus.utils.uniform_sampler import UniformWithReplacementSampler
from torch.utils.data import DataLoader
import numpy as np
from opacus import PrivacyEngine
from opacus.utils.batch_memory_manager import BatchMemoryManager
from torch import nn
from opacus.grad_sample.grad_sample_module import create_or_accumulate_grad_sample
from opacus.data_loader import DPDataLoader
from opacus.grad_sample import GradSampleModule
import wandb
#import optuna
from functools import partial
import numpy as np
from imgaug import augmenters as iaa
import os
import random
from torch.utils.data import ConcatDataset, DataLoader, random_split

In [ ]:
class Modified_Deep_Emotion(nn.Module):
  def __init__(self):
    '''
    Deep_Emotion class contains the network architecture.
    '''
    super(Modified_Deep_Emotion,self).__init__()
    self.conv1 = nn.Conv2d(1,10,3)
    self.conv2 = nn.Conv2d(10,10,3)
    self.pool2 = nn.MaxPool2d(2,2)

    self.conv3 = nn.Conv2d(10,10,3)
    self.conv4 = nn.Conv2d(10,10,3)
    self.pool4 = nn.MaxPool2d(2,2)

    self.norm = nn.BatchNorm2d(10)

    self.fc1 = nn.Linear(112360, 50)  # 更新这一行
    self.fc2 = nn.Linear(50,5)
    self.dropout = nn.Dropout(p=0.5)  # Dropout層

    self.localization = nn.Sequential(
      nn.Conv2d(1, 8, kernel_size=7),#既然输入图像不是灰度图，我假设它是一个RGB图像。这意味着输入图像应该有3个通道。
                                           #nn.Conv2d(1, 8, kernel_size=7),
      nn.MaxPool2d(2, stride=2),
      nn.ReLU(True),
      nn.Conv2d(8, 10, kernel_size=5),
      nn.MaxPool2d(2, stride=2),
      nn.ReLU(True)
        )

    self.fc_loc = nn.Sequential(
      nn.Linear(27040, 32),
      nn.ReLU(True),
      nn.Linear(32, 3 * 2)
        )
    self.fc_loc[2].weight.data.zero_()
    self.fc_loc[2].bias.data.copy_(torch.tensor([1, 0, 0, 0, 1, 0], dtype=torch.float))

  def stn(self, x):
    xs = self.localization(x)
    #print("Shape of xs before reshaping:", xs.shape)
    # 获取xs的形状
    batch_size, channels, height, width = xs.shape

    # 重塑xs
    xs = xs.view(batch_size, -1)

    # 再次打印xs的形状以确认重塑是否正确
    #print("Shape of xs after reshaping:", xs.shape)
    theta = self.fc_loc(xs)
    theta = theta.view(-1, 2, 3)

    grid = F.affine_grid(theta, x.size())
    x = F.grid_sample(x, grid)
    return x

  def forward(self,input):
    #print("Input shape to Deep_Emotion:", input.shape)  # Debug print
    out = self.stn(input)
    #print("Output shape after STN:", out.shape)  # Debug print

    out = F.relu(self.conv1(out))
    #print("Output shape after conv1:", out.shape)  # Debug print

    out = self.conv2(out)
    #print("Output shape after conv2:", out.shape)  # Debug print
    out = self.dropout(out)
    out = F.relu(self.pool2(out))
    #print("Output shape after pool2:", out.shape)  # Debug print

    out = F.relu(self.conv3(out))
    out = self.dropout(out)
    out = self.norm(out)
    out = F.relu(self.conv4(out))  # 使用conv4层
    #print("Output shape after conv3:", out.shape)  # Debug print

    out = F.dropout(out)

    # 获取out的形状
    batch_size, num_features, height, width = out.shape

    # 使用这些维度来计算每个输入样本的特征数量
    num_features_total = num_features * height * width

    # 使用计算出的特征数量来更新view函数的参数
    out = out.view(batch_size, num_features_total)
    out = F.relu(self.fc1(out))
    out = self.fc2(out)

    return out

In [ ]:
class CombinedModel(nn.Module):
  def __init__(self, num_classes):  # Set default value of num_classes to 5
    super(CombinedModel, self).__init__()

    # ResNet-18 model
    self.resnet18 = model_resnet
    # Remove the last layer to get features
    modules = list(self.resnet18.children())[:-1]
    self.resnet18 = nn.Sequential(*modules)

    # Modified_Deep_Emotion model
    self.deep_emotion = Modified_Deep_Emotion()

    # Fully connected layer
    self.fc = nn.Linear(517, num_classes)  # 512 from ResNet-18 and 53 from Modified_Deep_Emotion

  def forward(self, x):
    x1 = self.resnet18(x)
    x1 = x1.view(x1.size(0), -1)

    # Convert the input image to grayscale
    x_gray = x.mean(dim=1, keepdim=True)

    x2 = self.deep_emotion(x_gray)
    x2 = x2.view(x2.size(0), -1)

    # Concatenate features from both models
    x = torch.cat((x1, x2), dim=1)

    x = self.fc(x)
    return x

In [ ]:
def replace_batchnorm(module: nn.Module) -> nn.Module:
  """
  Recursively replace all BatchNorm2d layers with GroupNorm in the given module.
  """
  # Base case: if module is a BatchNorm2d, replace it with GroupNorm
  if isinstance(module, nn.BatchNorm2d):
    num_channels = module.num_features
    return nn.GroupNorm(1, num_channels)

  # Recursive case: if the module contains child modules, replace in each child
  for name, child_module in module.named_children():
    module.add_module(name, replace_batchnorm(child_module))

  return module

seq = iaa.Sequential([
    iaa.Fliplr(0.5), # horizontal flips
    iaa.Crop(percent=(0, 0.1)), # random crops
    iaa.Sometimes(0.5,
        iaa.GaussianBlur(sigma=(0, 0.5))
    ),
    # Strengthen or weaken the contrast in each image.
    iaa.LinearContrast((0.75, 1.5)),

    iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05*255), per_channel=0.5),

    iaa.Multiply((0.8, 1.2), per_channel=0.2),

    iaa.Affine(
        scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},
        translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)},
        rotate=(-25, 25),
        shear=(-8, 8)
    )
], random_order=True) # apply augmenters in random order



In [ ]:
from PIL import Image

class ImgAugTransform:
    def __init__(self, augmenters):
        self.augmenters = augmenters

    def __call__(self, img):
        img = np.array(img)
        img = self.augmenters.augment_image(img)
        return Image.fromarray(img)

"""## load model"""

transform = transforms.Compose([
  ImgAugTransform(seq),
  #transforms.Lambda(lambda img: seq.augment_image(np.array(img))),
  transforms.Resize((224, 224)),
  transforms.ToTensor(),
  #transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
  #transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
  #transforms.RandomErasing(p=0.5, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False)
])

# 替换为您的数据集路径
large_dataset_path = "/content/kaggle_dataset_v2"
expw_path = "/content/Expw-F"

large_train_dataset = datasets.ImageFolder(os.path.join(large_dataset_path, "train"), transform=transform)
expw_train_dataset = datasets.ImageFolder(os.path.join(expw_path, "train"), transform=transform)

percent = 0.8
# 2. 创建混合数据集
# 从大型数据集中抽取30%的样本（您可以根据需要修改这个比例）
num_samples_large = len(large_train_dataset)
subset_indices = random.sample(range(num_samples_large), int(percent * num_samples_large))
subset_large_train_dataset = torch.utils.data.Subset(large_train_dataset, subset_indices)

# 将fer2013数据集与大型数据集的子集结合
combined_train_dataset = ConcatDataset([subset_large_train_dataset, expw_train_dataset])
# 为验证和测试创建混合数据集 (我们仅使用fer2013的数据)
combined_val_dataset = datasets.ImageFolder(os.path.join(expw_path, "val"), transform=transform)
combined_test_dataset = datasets.ImageFolder(os.path.join(expw_path, "test"), transform=transform)

# 创建数据加载器
train_loader = DataLoader(combined_train_dataset, batch_size=32, shuffle=True)
train_loader = DPDataLoader.from_data_loader(train_loader, distributed=False)

val_loader = DataLoader(combined_val_dataset, batch_size=32, shuffle=False)
val_loader = DPDataLoader.from_data_loader(val_loader, distributed=False)

test_loader = DataLoader(combined_test_dataset, batch_size=32, shuffle=False)
test_loader = DPDataLoader.from_data_loader(test_loader, distributed=False)

In [ ]:
"""## load res 18 model"""

model_resnet = models.resnet18(pretrained=True)
num_features_for_combining = 512  # This is the default number of features from ResNet18
model_resnet.fc = nn.Linear(model_resnet.fc.in_features, num_features_for_combining)

"""## combined and replace the bnn"""

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = CombinedModel(num_classes=4)  # Replace YOUR_NUM_CLASSES with the number of classes in your dataset
#model = model.to(device)

model = replace_batchnorm(model)
model = model.to(device)



/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [ ]:
"""## load parameter"""

# Define loss and optimizer
lr = 0.00005

criterion = nn.CrossEntropyLoss()

#optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
#optimizer = optim.Adam(model.parameters(), lr=0.001, betas=(0.9, 0.999), eps=1e-08)
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, betas=(0.9, 0.99), weight_decay = 0.001)
# 初始化ReduceLROnPlateau调度器
scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.1, patience=5, verbose=True)
# Initialize the early stopping object
#early_stopping = EarlyStopping(patience=7, verbose=True)

wandb.login(key="")

wandb: WARNING Calling wandb.login() after wandb.init() has no effect.


True

In [ ]:




"""## Step1: befere add noise"""

import wandb
epochs = 100
# Initialize wandb
wandb.init(project="deep_emotion_project(Expw)", name="(data aug two (80%)combine_datasets)two_model_before_add_noise_run")

# Log hyperparameters
wandb.config.update({
    "epochs": epochs,
    "lr": lr,
    "optimizer": "AdamW",
})

def Train(epochs,train_loader,val_loader,criterion,optmizer,device):
    '''
    Training Loop
    '''
    print("===================================Start Training===================================")
    for epoch in range(epochs):
        train_loss = 0
        validation_loss = 0
        train_correct = 0
        val_correct = 0
        # Train the model  #
        model.train()
        for data, labels in train_loader:
            data, labels = data.to(device), labels.to(device)
            optmizer.zero_grad()
            outputs = model(data)
            loss = criterion(outputs,labels)
            loss.backward()
            optmizer.step()
            train_loss += loss.item()
            _, preds = torch.max(outputs,1)
            train_correct += torch.sum(preds == labels.data)

        train_loss = train_loss / len(train_loader.dataset)
        train_acc = train_correct.double() / len(train_loader.dataset)
        # Log training metrics
        wandb.log({"Train Loss(100%)": 100 * train_loss, "Train Accuracy(100%)": 100 * train_acc})


        #validate the model#
        model.eval()
        for data,labels in val_loader:
            data, labels = data.to(device), labels.to(device)
            val_outputs = model(data)
            val_loss = criterion(val_outputs, labels)
            validation_loss += val_loss.item()
            _, val_preds = torch.max(val_outputs,1)
            val_correct += torch.sum(val_preds == labels.data)
        # Evaluation on test data
        validation_loss = validation_loss / len(val_loader.dataset)
        val_acc = val_correct.double() / len(val_loader.dataset)

        # Log validation metrics
        wandb.log({"Validation Loss(100%)": 100 * validation_loss, "Validation Accuracy(100%)": 100* val_acc})

        # Step the scheduler
        scheduler.step(validation_loss)
        print('Epoch: {} \tTraining Loss: {:.8f}% \tValidation Loss {:.8f}% \tTraining Acuuarcy {:.3f}% \tValidation Acuuarcy {:.3f}%'
                                                           .format(epoch+1, 100 * train_loss,100 * validation_loss,train_acc * 100, val_acc*100))


    torch.save(model, 'deep_emotion-{}.pth'.format(epochs))
    print("===================================Training Finished===================================")

epochs = 100

Train(epochs, train_loader, val_loader, criterion, optimizer, device)

===================================Start Training===================================


/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4358: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4296: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4358: UserWarning: Default grid_sample and affine_grid behavior has changed to align_corners=False since 1.3.0. Please specify align_corners=True if the old behavior is desired. See the documentation of grid_sample for details.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:4296: UserWar

Epoch: 1 	Training Loss: 3.80958235% 	Validation Loss 3.54089591% 	Training Acuuarcy 47.959% 	Validation Acuuarcy 54.642%
Epoch: 2 	Training Loss: 3.75512517% 	Validation Loss 3.49766635% 	Training Acuuarcy 49.793% 	Validation Acuuarcy 55.935%
Epoch: 3 	Training Loss: 3.72453044% 	Validation Loss 3.61630049% 	Training Acuuarcy 50.239% 	Validation Acuuarcy 53.504%
Epoch: 4 	Training Loss: 3.72513700% 	Validation Loss 3.54623701% 	Training Acuuarcy 49.913% 	Validation Acuuarcy 54.616%
Epoch: 5 	Training Loss: 3.67030533% 	Validation Loss 3.49238768% 	Training Acuuarcy 50.457% 	Validation Acuuarcy 54.642%
Epoch: 6 	Training Loss: 3.64148106% 	Validation Loss 3.42994182% 	Training Acuuarcy 50.573% 	Validation Acuuarcy 56.426%
Epoch: 7 	Training Loss: 3.57120601% 	Validation Loss 3.38260789% 	Training Acuuarcy 51.225% 	Validation Acuuarcy 55.831%
Epoch: 8 	Training Loss: 3.45962439% 	Validation Loss 3.29003486% 	Training Acuuarcy 53.754% 	Validation Acuuarcy 57.073%
Epoch: 9 	Training Loss: